In [0]:
dbutils.widgets.removeAll()

In [0]:
catalogs = [row.catalog for row in spark.sql("SHOW CATALOGS").collect()]
catalogs.sort()
if catalogs:
    dbutils.widgets.combobox("catalog", catalogs[0], catalogs)
else:
    dbutils.widgets.combobox("catalog", "", [])

In [0]:
catalog = dbutils.widgets.get("catalog")
schemas = [row.databaseName for row in spark.sql(f"SHOW SCHEMAS IN {catalog}").collect()] if catalog else []
schemas.sort()
if schemas:
    dbutils.widgets.combobox("schema", schemas[0], schemas)
else:
    dbutils.widgets.combobox("schema", "", [])

In [0]:
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")
def drop_history_tables(spark, catalog, schema):
    tables = spark.sql(f"SHOW TABLES IN {catalog}.{schema}").filter("isTemporary = false")
    table_names = [row.tableName for row in tables.collect()]
    for name in table_names:
        if name.endswith("_file_version_history") or name.endswith("_transaction_history"):
            spark.sql(f"DROP TABLE IF EXISTS {catalog}.{schema}.{name}")

drop_history_tables(spark, catalog, schema)